**Imports**

In [1]:
import pandas as pd
import os

**Paths**

In [2]:
MAPPINGS_DATA_PATH = "../data/tmp/mappings"
INPUT_DATA_PATH = "../data/tmp/input"
OUTPUT_DATA_PATH = "../data/tmp/output"

**Suplementary Functions**

In [3]:
def assign_sector(sST):
    if sST.startswith("sST_DSTRA"):
        return "Transportation"
    elif sST.startswith("sST_DSIND"):
        return "Industry"
    elif sST.startswith("sST_DSOTH"):
        return "Residential and Commercial"
    else:
        return "Other"

**Data Preprocessing**

In [4]:
sPE = pd.read_csv(MAPPINGS_DATA_PATH + "/sPE_mapping.csv")
SPE_dict = dict(zip(sPE['SPE'].values, sPE['DESCRIPTION'].values))

vQPEDom = pd.read_csv(os.path.join(OUTPUT_DATA_PATH,"vQPEDom.csv"))
vQPEDom.rename(columns={"vQPEDom": "value"}, inplace=True)
vQPEDom['Description'] = vQPEDom['sPE'].map(SPE_dict)
vQPEDom['sYear'] = vQPEDom['sYear'].str[1:]

vQPEImp = pd.read_csv(os.path.join(OUTPUT_DATA_PATH,"vQPEImp.csv"))
vQPEImp.rename(columns={"vQPEImp": "value"}, inplace=True)
vQPEImp['Description'] = vQPEImp['sPE'].map(SPE_dict)
vQPEImp['sYear'] = vQPEImp['sYear'].str[1:]

In [5]:
vEmiCO2ST = pd.read_csv(os.path.join(OUTPUT_DATA_PATH,"vEmiCO2ST.csv"))
vEmiCO2ST.rename(columns={"vEmiCO2ST":"value"}, inplace=True)
vEmiCO2ST['Sector'] = vEmiCO2ST['sST'].apply(assign_sector)
vEmiCO2ST = vEmiCO2ST[["sYear", "Sector", "value"]]
vEmiCO2ST['sYear'] = vEmiCO2ST['sYear'].str[1:]

vEmiCO2CE = pd.read_csv(os.path.join(OUTPUT_DATA_PATH,"vEmiCO2CE.csv"))
vEmiCO2CE['Sector'] = "Generation"
vEmiCO2CE.rename(columns={"vEmiCO2CE":"value"}, inplace=True)
vEmiCO2CE = vEmiCO2CE[["sYear", "Sector", "value"]]
vEmiCO2CE['sYear'] = vEmiCO2CE['sYear'].str[1:]

In [6]:
vQSTInTE = pd.read_csv(os.path.join(OUTPUT_DATA_PATH,"vQSTInTE.csv"))
vQSTInTE.rename(columns={"vQSTInTE":"value"}, inplace=True)
vQSTInTE['Sector'] = vQSTInTE['sST'].apply(assign_sector)
vQSTInTE = vQSTInTE[["sYear", "Sector", "value"]]
vQSTInTE['sYear'] = vQSTInTE['sYear'].str[1:]
vQSTInTE = vQSTInTE.groupby(['sYear', 'Sector']).sum()['value'].reset_index()
vQSTInTE['value'] = vQSTInTE['value']/1000

# Barchart

**Imports**

In [7]:
from charts.barchart import create_stacked_bar_chart

**Data Processing**

In [8]:
primary_energies_grouped = pd.concat([vQPEDom, vQPEImp]).groupby(['Description', 'sYear']).sum('value').reset_index()
primary_energies_grouped['value'] = primary_energies_grouped['value']/1000
df = primary_energies_grouped.copy()

**Plot**

In [9]:
create_stacked_bar_chart(df, 'sYear', 'value', 'Description', title='Primary Energy Consumption', x_axis_title='Year', y_axis_title= "Energy (GWh)", plot_bg_color='white')

In [10]:
#vQSTInTE = vQSTInTE[vQSTInTE['sYear'] < "2031"]
#vQSTInTE = vQSTInTE[~(vQSTInTE['Sector'] == "Industry")]

In [11]:
create_stacked_bar_chart(vQSTInTE, 'sYear', 'value', 'Sector', title='Final Energy Consumption', x_axis_title='Year', y_axis_title= "Energy (GWh)", plot_bg_color='white')

# Pie Chart

**Imports**

In [12]:
from charts.piechart import process_data
from charts.piechart import create_donut_chart

**Data Preprocessing**

In [13]:
# Process data
grouped_df = process_data(vQPEDom, ['Description', 'sYear'], 'value')

**Plot**

In [14]:
# Plot for a specific year
create_donut_chart(grouped_df, 2025, 'value', 'Description', title_text="Distribution of Renewable Energy Consumption for Year {}", 
                   center_text="{}", hole_size=0.45, font_size=25)

# Area Chart

**Imports**

In [15]:
from charts.areachart import create_stacked_area_chart

**Data Preprocessing**

In [16]:
# Concatenate the dataframes
df_combined = pd.concat([vEmiCO2CE, vEmiCO2ST])

# Group by 'sYear' and 'Sector' and sum 'value'
df_grouped = df_combined.groupby(['sYear', 'Sector'])['value'].sum().reset_index()
df_grouped['value'] = df_grouped['value'] / 1000
#df_grouped = df_grouped[~((df_grouped['Sector'] == "Industry") | (df_grouped['Sector'] == "Generation"))]
#df_grouped = df_grouped[df_grouped['sYear'] < "2031"]

**Plot**

In [17]:
create_stacked_area_chart(df_grouped, 'sYear', 'value', 'Sector', title='Sector-wise CO2 Emission Over Time', y_axis_title="CO2 Emissions (Mt)", x_axis_title='Year', barmode='stack')

In [18]:
#primary_energies_grouped = primary_energies_grouped[primary_energies_grouped['sYear'] < "2031"]

In [19]:
create_stacked_area_chart(primary_energies_grouped, 'sYear', 'value', 'Description', title='Primary Energy Consumption', y_axis_title= "Energy (GWh)", x_axis_title='Year', barmode='stack')